In [304]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from mlxtend.feature_selection import ColumnSelector
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [305]:
# Displaying all columns
pd.set_option('display.max_columns', None)

# #Display all data in columns
# pd.set_option("display.max_colwidth", None)

# np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.expand_frame_repr", True)
# pd.set_option('display.width', 1000)


In [306]:
#Storing Accuracy

data_accuracy = pd.DataFrame(columns=['Algo', 'Accuracy(KFold)', 'Test', 'Train', 'Recall', 'Precision', 'F1', 'ROC_AUC_SCORE'])
def store(data):
    data_accuracy.loc[len(data_accuracy.index)] = data
    data_accuracy.to_csv('loan_prediction_metric_scores.csv')
    print(data_accuracy)



def get_info_pipeline(pipe_name):
    sub_pipe = pipe_name[:-1]
    transformed_df = pd.DataFrame(
        data=sub_pipe.fit_transform(X_train, y_train),
        columns=sub_pipe.get_feature_names_out(),
    )
    print(transformed_df)

In [307]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [308]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [309]:
df1.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [310]:
#Renaming the 'movie_id' col in df1 to 'id' of df2 

In [311]:
df1.rename(columns = {'movie_id':'id'}, inplace = True)

In [312]:
df1.head(1)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [313]:
#joining the credits amd movie dataframes

In [314]:
df2 = pd.merge(df1, df2, on='id')

In [315]:
df2.head(1)

,id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [316]:
#Drpping columns

In [317]:
df2.columns

Index(['id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count'],
      dtype='object')

In [318]:
df2.drop(columns=['status', 'runtime'], axis=1, inplace=True)

In [319]:
df2.drop(columns=['title_x'], axis=1, inplace=True)

In [320]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,title_y,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800


In [321]:
df2.drop(columns=['title_y'], axis=1, inplace=True)

In [322]:
df2.shape

(4803, 19)

In [323]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800


In [324]:
df2.drop(columns=['revenue', 'release_date'], axis=1, inplace=True)

In [325]:
df2.drop(columns=['budget', 'homepage', 'popularity', 'spoken_languages', 'production_countries', 'vote_average', 'vote_count'], axis=1, inplace=True)

In [326]:
df2.shape

(4803, 10)

In [327]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,production_companies,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.


In [328]:
df2.isnull().sum()

id                        0
cast                      0
crew                      0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  3
production_companies      0
tagline                 844
dtype: int64

In [329]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4803 non-null   int64 
 1   cast                  4803 non-null   object
 2   crew                  4803 non-null   object
 3   genres                4803 non-null   object
 4   keywords              4803 non-null   object
 5   original_language     4803 non-null   object
 6   original_title        4803 non-null   object
 7   overview              4800 non-null   object
 8   production_companies  4803 non-null   object
 9   tagline               3959 non-null   object
dtypes: int64(1), object(9)
memory usage: 412.8+ KB


In [330]:
df2.drop(columns='production_companies', axis=1, inplace=True)

In [331]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.


In [332]:
df2.shape

(4803, 9)

In [333]:
#Dropping null rows

In [334]:
df2.isnull().sum()

id                     0
cast                   0
crew                   0
genres                 0
keywords               0
original_language      0
original_title         0
overview               3
tagline              844
dtype: int64

In [335]:
df2.drop(columns='tagline', axis=1, inplace=True)

In [336]:
df2.dropna(inplace=True)

In [337]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [338]:
df2.isnull().sum()

id                   0
cast                 0
crew                 0
genres               0
keywords             0
original_language    0
original_title       0
overview             0
dtype: int64

In [339]:
#Preprocessing

In [340]:
##Genre

In [341]:
df2.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [342]:
import ast

In [343]:
def fetch_genres(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
    return ls

In [344]:
df2['genres'] = df2['genres'].apply(fetch_genres)

In [345]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [346]:
df2.iloc[3, 3]

['Action', 'Crime', 'Drama', 'Thriller']

In [347]:
##Crew

In [348]:
def fetch_dir_and_prod(string):
    ls = []
    for i in  ast.literal_eval(string):
        if i['job']=='Director':
            ls.append(i['name'])
            break
    return ls

In [349]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [350]:
msg = " "
msg = df2.loc[4].crew

In [351]:
msg

'[{"credit_id": "52fe479ac3a36847f813eaa3", "department": "Writing", "gender": 2, "id": 7, "job": "Screenplay", "name": "Andrew Stanton"}, {"credit_id": "52fe479ac3a36847f813ea65", "department": "Directing", "gender": 2, "id": 7, "job": "Director", "name": "Andrew Stanton"}, {"credit_id": "58c69e819251411d30017673", "department": "Crew", "gender": 2, "id": 7879, "job": "Thanks", "name": "John Lasseter"}, {"credit_id": "52fe479ac3a36847f813eac5", "department": "Production", "gender": 2, "id": 490, "job": "Producer", "name": "Colin Wilson"}, {"credit_id": "58c68f2b9251411d30016cf2", "department": "Production", "gender": 2, "id": 2073, "job": "Casting", "name": "Gail Stevens"}, {"credit_id": "58c68ef19251411d62016ec3", "department": "Sound", "gender": 2, "id": 3996, "job": "Sound Re-Recording Mixer", "name": "Tom Johnson"}, {"credit_id": "52fe479ac3a36847f813ead7", "department": "Editing", "gender": 2, "id": 6958, "job": "Editor", "name": "Eric Zumbrunnen"}, {"credit_id": "58c69a9ec3a3684

In [352]:
import ast
msg = ast.literal_eval(msg)

In [353]:
for i in msg:
    if i['job']=='Director':
        print(i['name'])
        break

Andrew Stanton


In [354]:
df2['crew'] = df2['crew'].apply(fetch_dir_and_prod)

In [355]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [356]:
##keywords

In [357]:
msg = df2.iloc[0, 4]

In [358]:
msg

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [359]:
def fetch_keyowrds(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
            
    return ls

In [360]:
fetch_keyowrds(msg)

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [361]:
df2['keywords'] = df2['keywords'].apply(fetch_genres)

In [362]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [363]:
##cast (first 3)

In [364]:
msg = df2.iloc[0, 1]

In [365]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [366]:
def fetch_crew_first_three(string):
    ls = []
    cnt = 3
    for i in  ast.literal_eval(string):
        if cnt==0:
            break
        else:
            ls.append(i['name'])
        cnt=cnt-1
    return ls

In [367]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [368]:
fetch_crew_first_three(msg)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [369]:
df2['cast'] = df2['cast'].apply(fetch_crew_first_three)

In [370]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [371]:
df2.rename(columns = {'crew':'director'}, inplace = True)

In [372]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [373]:
#Preprocessing 'overview' column

#Lowercasing

df2['overview'] = df2['overview'].str.lower()


# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_html)
# print(df.head())


#Remove @

def remove_at_the_rate(text):

  ls = []
  new = []

  ls = nlp(text)

  for word in ls:
        if word.text != "@":
            new.append(word.text)

  return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_at_the_rate)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_url)
# print(df.head())


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

df2['overview'] = df2['overview'].apply(remove_punc)


#Correcting spelling

# from textblob import TextBlob

# def spell_check(text):

#     textblb = TextBlob(text)
#     return textblb.correct().string

# X['remainder__text'] = X['remainder__text'].apply(spell_check)


from autocorrect import Speller

check = Speller()

def check_spell(text):

  return check(text)

df2['overview'] = df2['overview'].apply(check_spell)


# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_stop_words)


#Removing Contradictions

import contractions

def remove_contradictions(text):

  return " ".join([contractions.fix(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(remove_contradictions)


# #Lemmetization

# from nltk.stem.wordnet import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()

def Lemmetization(text):

  return " ".join([word.lemma_ for word in nlp(text)])


# #Stemming

# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()
# def stemming(text):

#     return " ".join([ps.stem(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(Lemmetization)




In [374]:
#Tokenization of overview column

In [375]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# def token_nlp(string):
#     ls = []
#     # doc1 = nlp(string)
#     doc1 = word_tokenize(string)
#     ls.append(doc1)
#     return ls

In [376]:
# df2['overview'] = df2['overview'].apply(token_nlp)

In [377]:
df2.iloc[0, 7]

'22nd century     paraplegic marine dispatch moon pandora unique mission     become torn follow order protect alien civilization'

In [378]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [379]:
df2['tags'] = df2.sum(axis=1)

In [380]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,tags
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,19995


In [381]:
#Removing spaces between names in cast and crew column

In [382]:
def remove_spaces(ls):
    for i in range(len(ls)):
        ls[i] = ls[i].replace(' ', '')
    return ls
        # print(ls[i])

In [383]:
ls1 = df2.iloc[0, 1]

In [384]:
ls1

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [385]:
remove_spaces(ls1)

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']

In [386]:
df2.drop(columns='tags', axis=1, inplace=True)

In [387]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [388]:
df2['cast'] = df2['cast'].apply(remove_spaces)

In [389]:
df2['director'] = df2['director'].apply(remove_spaces)

In [390]:
df2['genres'] = df2['genres'].apply(remove_spaces)

In [394]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [392]:
df2['keywords'] = df2['keywords'].apply(remove_spaces)

In [395]:
df2['combined'] = df2['cast'] + df2['director'] + df2['genres'] + df2['keywords'] 

In [396]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch mo...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."


In [397]:
msg = df2.iloc[0, 7]

In [398]:
df2.iloc[0, 8]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [399]:
def replace_big_spaces(string):
    string = string.replace('  ', '')
    return string

In [400]:
replace_big_spaces(msg)

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [401]:
df2['overview'] = df2['overview'].apply(replace_big_spaces)

In [402]:
df2.head(5)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,captain barbossa long believe dead come back l...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",en,Spectre,cryptic message bond ’s past send trail uncove...,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",en,The Dark Knight Rises,follow death district attorney harvey dent bat...,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",en,John Carter,john carter war weary former military captain ...,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [403]:
msg = df2.iloc[0, 7]

In [404]:
msg

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [405]:
def list_to_string(ls):
    st=""
    
    for i in ls:
        st = ' '.join(ls)
    return st

In [406]:
msg = df2.iloc[0, 8]

In [407]:
msg

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [408]:
list_to_string(msg)

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [409]:
df2['combined'] = df2['combined'].apply(list_to_string)

In [410]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [417]:
df2['tags'] = df2['combined'] + df2['overview']

In [418]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [419]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [420]:
msg = df2.iloc[0, 5]

In [421]:
msg

'en'

In [423]:
msg = ' ' +  msg

In [424]:
msg

' en'

In [425]:
def insert_space_at_beginning(string):
    string = ' ' + string
    return string

In [426]:
df2['original_language'] = df2['original_language'].apply(insert_space_at_beginning)

In [427]:
df2['tags'] = df2['tags'] + df2['original_language']

In [428]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [429]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'